In [2]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [4]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

In [12]:
r.X #X at the row level

-77.0533213960144

In [14]:
df.X #X of whole data frame

0       -77.033293
1       -77.005401
2       -76.949648
3       -77.027959
4       -76.945923
           ...    
27926   -76.974974
27927   -77.031955
27928   -77.020926
27929   -77.001328
27930   -77.053321
Name: X, Length: 27931, dtype: float64

## Warm-up Demo

In [38]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

CPU times: user 663 ms, sys: 12.8 ms, total: 675 ms
Wall time: 712 ms


In [40]:
%%time
for i,r in df.iterrows(): #iterrows returns a tuple, so need i
    r.X + r.Y

CPU times: user 416 ms, sys: 13.6 ms, total: 430 ms
Wall time: 447 ms


In [45]:
%%time
df.apply(lambda r: r.X + r.Y, axis = 1)

CPU times: user 335 ms, sys: 6.79 ms, total: 342 ms
Wall time: 339 ms


0       -38.064623
1       -38.122138
2       -38.050995
3       -38.076144
4       -38.063962
           ...    
27928   -38.135656
27929   -38.118390
27930   -38.089127
27931   -38.099146
27932   -38.089438
Length: 27933, dtype: float64

In [44]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

CPU times: user 3.2 ms, sys: 2.78 ms, total: 5.98 ms
Wall time: 6.9 ms


0       -38.064623
1       -38.122138
2       -38.050995
3       -38.076144
4       -38.063962
           ...    
27928   -38.135656
27929   -38.118390
27930   -38.089127
27931   -38.099146
27932   -38.089438
Length: 27933, dtype: float64

# Practice

In [26]:
df.CCN

0        20118678
1        20023507
2        20090091
3        20048262
4        20050930
           ...   
27926    20013962
27927    20015078
27928    20157380
27929    20024376
27930    20089061
Name: CCN, Length: 27931, dtype: int64

In [44]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250'] #these are string so that is why you need to convert to strings in next line of code
C_Tar = C_Target = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']]
C_Oth = C_Other  = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

,CCN,WARD,OFFENSE,report_dt
941,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00
14013,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00


In [42]:
C_Oth

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,report_dt
0,-77.033293,38.919188,20118678,2020/08/20 00:24:28+00,EVENING,OTHERS,THEFT/OTHER,1400 - 1499 BLOCK OF W STREET NW,397113.00,139035.00,...,4300.0,Precinct 22,38.919180,-77.033291,NaN,2020/08/08 00:24:44+00,2020/08/20 00:24:46+00,596090405,NaN,2020-08-20 00:24:28+00:00
1,-77.005401,38.879914,20023507,2020/02/08 05:59:18+00,MIDNIGHT,OTHERS,MOTOR VEHICLE THEFT,800 - 899 BLOCK OF NEW JERSEY AVE SE,399531.54,134674.78,...,7203.0,Precinct 131,38.879907,-77.005399,CAPITOL RIVERFRONT,2020/02/08 01:43:11+00,2020/02/08 01:53:11+00,596090413,NaN,2020-02-08 05:59:18+00:00
2,-76.949648,38.865420,20090091,2020/06/19 09:34:15+00,MIDNIGHT,OTHERS,THEFT/OTHER,1800 - 1899 BLOCK OF FORT DAVIS STREET SE,404370.00,133067.00,...,9902.0,Precinct 110,38.865412,-76.949646,NaN,2020/06/16 15:00:02+00,2020/06/18 16:00:22+00,596090448,NaN,2020-06-19 09:34:15+00:00
3,-77.027959,38.961979,20048262,2020/03/20 04:20:47+00,MIDNIGHT,OTHERS,THEFT F/AUTO,5900 - 5999 BLOCK OF GEORGIA AVENUE NW,397577.00,143785.00,...,1901.0,Precinct 58,38.961971,-77.027957,NaN,2020/03/19 23:22:40+00,2020/03/20 00:49:00+00,596090460,NaN,2020-03-20 04:20:47+00:00
4,-76.945923,38.900875,20050930,2020/03/26 04:28:40+00,MIDNIGHT,OTHERS,THEFT F/AUTO,740 - 899 BLOCK OF KENILWORTH AVENUE NE,404691.00,137003.00,...,9602.0,Precinct 100,38.900868,-76.945921,NaN,2020/03/25 20:00:23+00,2020/03/26 04:10:44+00,596090461,NaN,2020-03-26 04:28:40+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27926,-76.974974,38.868554,20013962,2020/01/24 04:18:43+00,MIDNIGHT,OTHERS,THEFT/OTHER,1712 - 1731 BLOCK OF 22ND STREET SE,402172.00,133414.00,...,7601.0,Precinct 133,38.868546,-76.974972,NaN,2020/01/24 00:11:14+00,2020/01/24 00:56:28+00,596642763,NaN,2020-01-24 04:18:43+00:00
27927,-77.031955,38.918648,20015078,2020/01/25 19:21:29+00,DAY,OTHERS,THEFT/OTHER,2100 - 2199 BLOCK OF 14TH STREET NW,397229.00,138975.00,...,4401.0,Precinct 22,38.918640,-77.031953,NaN,2020/01/25 18:35:51+00,2020/01/25 18:50:11+00,596642764,NaN,2020-01-25 19:21:29+00:00
27928,-77.020926,38.899814,20157380,2020/11/03 20:43:43+00,EVENING,OTHERS,THEFT/OTHER,600 - 699 BLOCK OF H STREET NW,398185.00,136884.00,...,5801.0,Precinct 129,38.899806,-77.020924,DOWNTOWN,2020/11/03 19:20:06+00,2020/11/03 20:43:53+00,596642770,NaN,2020-11-03 20:43:43+00:00
27929,-77.001328,38.892843,20024376,2020/02/09 19:41:19+00,DAY,OTHERS,THEFT F/AUTO,300 - 399 BLOCK OF MARYLAND AVENUE NE,399885.00,136110.00,...,8200.0,Precinct 84,38.892836,-77.001326,CAPITOL HILL,2020/02/09 15:00:04+00,2020/02/09 19:41:20+00,596642783,NaN,2020-02-09 19:41:19+00:00


**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [88]:
#shift time to find all possible parameters
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as shape #generates a sequence of integers from 0 to C_Tar.shape[0] - 1
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20165648 279   -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
         1308  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         4675  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00   
         11621 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         15616 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 279     EVENING  OTHERS  MOTOR VEHICLE THEFT   
         1308   MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         4675        DAY  OTHERS         THEFT F/AUTO   
         11621   EVENING  OTHERS          THEFT/OTHER   
         15616       DAY  OTHERS          THEFT/OTHER   

                                                     BLOCK    XBLOCK  \
20165648 279            1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
         1308             100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         4675          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         11621  300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         15616   600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 279    137478.0  ...       10603.0    Precinct 144  38.905159   
         1308   135959.0  ...        8200.0     Precinct 89  38.891475   
         4675   137456.0  ...       10602.0     Precinct 83  38.904961   
         11621  136899.0  ...        5900.0    Precinct 143  38.899942   
         15616  135255.0  ...        6500.0     Precinct 89  38.885133   

                LONGITUDE           BID              START_DATE  \
20165648 279   -77.005891          NOMA  2020/11/20 20:15:26+00   
         1308  -76.999516           NaN  2020/11/20 03:02:27+00   
         4675  -76.997314           NaN  2020/11/19 22:30:39+00   
         11621 -77.015552      DOWNTOWN  2020/11/20 17:30:16+00   
         15616 -76.997326  CAPITOL HILL  2020/11/19 23:43:15+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20165648 279    2020/11/20 21:46:24+00  596093043             NaN   
         1308                      NaN  596105470             NaN   
         4675   2020/11/20 03:00:43+00  596362501             NaN   
         11621  2020/11/20 22:08:28+00  596387673             NaN   
         15616                     NaN  596402829             NaN   

                               report_dt  
20165648 279   2020-11-20 22:17:27+00:00  
         1308  2020-11-20 04:27:36+00:00  
         4675  2020-11-20 15:06:04+00:00  
         11621 2020-11-20 22:07:10+00:00  
         15616 2020-11-20 12:46:32+00:00  

[5 rows x 26 columns]

# 1.5 Iterrow Approach

In [50]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 734   -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         9297  -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   
         14014 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         14015 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         18683 -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 734    EVENING  OTHERS  MOTOR VEHICLE THEFT   
         9297   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         14014      DAY  OTHERS         THEFT F/AUTO   
         14015      DAY  OTHERS         THEFT F/AUTO   
         18683      DAY  OTHERS          THEFT/OTHER   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 734         1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   
         9297        1100 - 1199 BLOCK OF F STREET NW  397609.0  136611.0   
         14014       1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         14015  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         18683       700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 734    ...       10700.0     Precinct 17  38.905648 -77.039822   
         9297   ...        5802.0    Precinct 129  38.897346 -77.027563   
         14014  ...        5303.0     Precinct 15  38.909638 -77.040089   
         14015  ...        4100.0     Precinct 13  38.913346 -77.050526   
         18683  ...        5801.0    Precinct 129  38.899121 -77.021926   

                            BID              START_DATE  \
20123250 734    GOLDEN TRIANGLE  2020/08/27 19:01:24+00   
         9297          DOWNTOWN  2020/08/29 23:08:57+00   
         14014              NaN  2020/08/28 22:00:23+00   
         14015              NaN  2020/08/26 22:00:29+00   
         18683         DOWNTOWN  2020/08/29 16:05:40+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 734    2020/08/29 19:00:05+00  596095073             NaN   
         9297                      NaN  596376108             NaN   
         14014  2020/08/29 08:00:27+00  596397251             NaN   
         14015  2020/08/27 12:00:51+00  596397252             NaN   
         18683  2020/08/29 16:08:33+00  596447637             NaN   

                               report_dt  
20123250 734   2020-08-29 22:04:46+00:00  
         9297  2020-08-30 00:05:52+00:00  
         14014 2020-08-29 16:05:18+00:00  
         14015 2020-08-29 16:45:57+00:00  
         18683 2020-08-29 17:15:19+00:00  

[5 rows x 26 columns]

## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

In [115]:
store_matches_2 = {}

def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    # Your code here
    same_ward = C_Oth[ C_Oth['WARD']==r['WARD'] ]
    
    same_ward_same_time = same_ward[ (same_ward['report_dt']>=r['report_dt']) &
                                    (same_ward['report_dt']<=(r['report_dt']+timedelta(minutes=1200)))].copy()
    #the previous same_ward_same_time is not a new dataframe, but a slice/view of the original dataframe. 

    store_matches_2[r['CCN']]= same_ward_same_time

In [117]:
r = C_Tar.iloc[0]

find_related_crimes(r)

### 2.2 apply it to one of the focal crimes

In [121]:
find_related_crimes(C_Tar.iloc[0])

In [72]:
# this is not a new data frame 
df1 = df[df.WARD == 1]
df1

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,report_dt
0,-77.033293,38.919188,20118678,2020/08/20 00:24:28+00,EVENING,OTHERS,THEFT/OTHER,1400 - 1499 BLOCK OF W STREET NW,397113.0,139035.0,...,4300.0,Precinct 22,38.919180,-77.033291,NaN,2020/08/08 00:24:44+00,2020/08/20 00:24:46+00,596090405,NaN,2020-08-20 00:24:28+00:00
7,-77.038004,38.930348,20120375,2020/08/23 14:20:13+00,DAY,OTHERS,THEFT/OTHER,3140 - 3172 BLOCK OF MOUNT PLEASANT STREET NW,396705.0,140274.0,...,2702.0,Precinct 39,38.930340,-77.038002,NaN,2020/08/23 13:50:59+00,2020/08/23 13:55:21+00,596090467,NaN,2020-08-23 14:20:13+00:00
12,-77.041588,38.925816,20401011,2020/07/21 18:40:02+00,DAY,OTHERS,THEFT/OTHER,2800 - 2899 BLOCK OF 18TH STREET NW,396394.0,139771.0,...,3901.0,Precinct 35,38.925808,-77.041586,NaN,2020/07/19 18:30:00+00,2020/07/20 00:00:00+00,596090485,NaN,2020-07-21 18:40:02+00:00
13,-77.046310,38.915409,20061534,2020/04/21 10:40:16+00,MIDNIGHT,OTHERS,BURGLARY,1811 - 1899 BLOCK OF CONNECTICUT AVENUE NW,395984.0,138616.0,...,4002.0,Precinct 25,38.915401,-77.046307,DUPONT CIRCLE,2020/04/21 09:10:18+00,2020/04/21 09:20:39+00,596090495,NaN,2020-04-21 10:40:16+00:00
16,-77.042956,38.917825,20401190,2020/08/02 00:09:54+00,EVENING,OTHERS,THEFT/OTHER,1800 - 1899 BLOCK OF CALIFORNIA STREET NW,396275.0,138884.0,...,4002.0,Precinct 25,38.917817,-77.042953,NaN,2020/08/01 22:30:00+00,2020/08/01 23:55:00+00,596090525,NaN,2020-08-02 00:09:54+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27904,-77.025199,38.920208,20080719,2020/06/01 04:46:56+00,MIDNIGHT,OTHERS,THEFT F/AUTO,2200 - 2299 BLOCK OF SHERMAN AVENUE NW,397815.0,139148.0,...,3500.0,Precinct 37,38.920200,-77.025196,NaN,2020/05/31 17:30:46+00,2020/06/01 02:50:05+00,596642611,NaN,2020-06-01 04:46:56+00:00
27910,-77.024218,38.918073,20178504,2020/12/17 08:41:49+00,MIDNIGHT,OTHERS,THEFT/OTHER,900 - 913 BLOCK OF FLORIDA AVENUE NW,397900.0,138911.0,...,4401.0,Precinct 137,38.918066,-77.024215,NaN,2020/12/17 08:00:36+00,2020/12/17 08:05:01+00,596642638,NaN,2020-12-17 08:41:49+00:00
27917,-77.040091,38.929690,20029123,2020/02/17 19:05:10+00,DAY,OTHERS,THEFT F/AUTO,1700 - 1799 BLOCK OF KENYON STREET NW,396524.0,140201.0,...,2702.0,Precinct 39,38.929682,-77.040089,NaN,2020/02/17 17:52:20+00,2020/02/17 17:53:26+00,596642688,NaN,2020-02-17 19:05:10+00:00
27920,-77.046026,38.923238,20174930,2020/12/09 21:38:05+00,EVENING,OTHERS,BURGLARY,1847 - 1999 BLOCK OF CALVERT STREET NW,396009.0,139485.0,...,4001.0,Precinct 25,38.923230,-77.046024,NaN,2020/12/04 19:35:02+00,2020/12/09 19:45:40+00,596642715,NaN,2020-12-09 21:38:05+00:00


In [84]:
# this is not a new data frame, it is a slice
df1 = df[df.WARD == 1]
df1

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,report_dt
0,-77.033293,38.919188,20118678,2020/08/20 00:24:28+00,EVENING,OTHERS,THEFT/OTHER,1400 - 1499 BLOCK OF W STREET NW,397113.0,139035.0,...,4300.0,Precinct 22,38.919180,-77.033291,NaN,2020/08/08 00:24:44+00,2020/08/20 00:24:46+00,596090405,NaN,2020-08-20 00:24:28+00:00
7,-77.038004,38.930348,20120375,2020/08/23 14:20:13+00,DAY,OTHERS,THEFT/OTHER,3140 - 3172 BLOCK OF MOUNT PLEASANT STREET NW,396705.0,140274.0,...,2702.0,Precinct 39,38.930340,-77.038002,NaN,2020/08/23 13:50:59+00,2020/08/23 13:55:21+00,596090467,NaN,2020-08-23 14:20:13+00:00
12,-77.041588,38.925816,20401011,2020/07/21 18:40:02+00,DAY,OTHERS,THEFT/OTHER,2800 - 2899 BLOCK OF 18TH STREET NW,396394.0,139771.0,...,3901.0,Precinct 35,38.925808,-77.041586,NaN,2020/07/19 18:30:00+00,2020/07/20 00:00:00+00,596090485,NaN,2020-07-21 18:40:02+00:00
13,-77.046310,38.915409,20061534,2020/04/21 10:40:16+00,MIDNIGHT,OTHERS,BURGLARY,1811 - 1899 BLOCK OF CONNECTICUT AVENUE NW,395984.0,138616.0,...,4002.0,Precinct 25,38.915401,-77.046307,DUPONT CIRCLE,2020/04/21 09:10:18+00,2020/04/21 09:20:39+00,596090495,NaN,2020-04-21 10:40:16+00:00
16,-77.042956,38.917825,20401190,2020/08/02 00:09:54+00,EVENING,OTHERS,THEFT/OTHER,1800 - 1899 BLOCK OF CALIFORNIA STREET NW,396275.0,138884.0,...,4002.0,Precinct 25,38.917817,-77.042953,NaN,2020/08/01 22:30:00+00,2020/08/01 23:55:00+00,596090525,NaN,2020-08-02 00:09:54+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27904,-77.025199,38.920208,20080719,2020/06/01 04:46:56+00,MIDNIGHT,OTHERS,THEFT F/AUTO,2200 - 2299 BLOCK OF SHERMAN AVENUE NW,397815.0,139148.0,...,3500.0,Precinct 37,38.920200,-77.025196,NaN,2020/05/31 17:30:46+00,2020/06/01 02:50:05+00,596642611,NaN,2020-06-01 04:46:56+00:00
27910,-77.024218,38.918073,20178504,2020/12/17 08:41:49+00,MIDNIGHT,OTHERS,THEFT/OTHER,900 - 913 BLOCK OF FLORIDA AVENUE NW,397900.0,138911.0,...,4401.0,Precinct 137,38.918066,-77.024215,NaN,2020/12/17 08:00:36+00,2020/12/17 08:05:01+00,596642638,NaN,2020-12-17 08:41:49+00:00
27917,-77.040091,38.929690,20029123,2020/02/17 19:05:10+00,DAY,OTHERS,THEFT F/AUTO,1700 - 1799 BLOCK OF KENYON STREET NW,396524.0,140201.0,...,2702.0,Precinct 39,38.929682,-77.040089,NaN,2020/02/17 17:52:20+00,2020/02/17 17:53:26+00,596642688,NaN,2020-02-17 19:05:10+00:00
27920,-77.046026,38.923238,20174930,2020/12/09 21:38:05+00,EVENING,OTHERS,BURGLARY,1847 - 1999 BLOCK OF CALVERT STREET NW,396009.0,139485.0,...,4001.0,Precinct 25,38.923230,-77.046024,NaN,2020/12/04 19:35:02+00,2020/12/09 19:45:40+00,596642715,NaN,2020-12-09 21:38:05+00:00


In [ ]:
# if you want to change the original, you should make a copy
df1 = df[df.WARD == 1].copy()
df1["boop"] = 1
df1

### 2.3 Use apply to cover all the other focal crimes